In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader

In [ ]:
import torchvision

In [ ]:
data_dir='/content/drive/MyDrive/Project'

In [ ]:
data_transforms={
    'train':transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),

    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    }

In [ ]:
image_datasets={x:datasets.ImageFolder(root=data_dir,transform=data_transforms[x])
                for x in ['train','val']}


In [ ]:
dataloaders={x:DataLoader(image_datasets[x],batch_size=2,shuffle=True,num_workers=4)
                for x in ['train','val']}

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class face_recognition(nn.Module):

  def __init__(self):
    super(face_recognition,self).__init__()
    self.model=models.resnet18(pretrained=True)
    num_ftrs=self.model.fc.in_features
    self.model.fc=nn.Linear(num_ftrs,2)

  def forward(self,x):
    x=self.model(x)
    return x

In [ ]:
model=face_recognition()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
criterion=nn.CrossEntropyLoss()

In [ ]:
optimizer=optim.SGD(model.parameters(),lr=0.001)

In [ ]:
def train_model(model,critetion,optimizer,num_epochs=10):
  for epoch in range(num_epochs):
    print(f'Epoch{epoch}/{num_epochs-1}')
    print('-'*10)

  for phase in ['train','val']:
    if phase == 'train':
      model.train()
    else:
      model.eval()
    running_loss=0.0
    running_corrects=0

    for inputs, labels in dataloaders[phase]:
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      with torch.set_grad_enabled(phase=='train'):
        outputs = model(inputs)
        _, preds = torch.max(outputs,1)
        loss = criterion(outputs, labels)

        if phase == 'train':
          loss.backward()
          optimizer.step()

        running_loss += loss.item()* inputs.size(0)
        running_corrects += torch.sum(preds==labels.data)
      epoch_loss=running_loss / len(image_datasets[phase])
      epoch_acc = running_corrects.double()/len(image_datasets[phase])

      print(f'{phase} Loss: {epoch_loss: .4f} Acc: {epoch_acc:.4f}')


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model=model.to(device)
train_model(model,criterion,optimizer,num_epochs=10)


Epoch0/9
----------
Epoch1/9
----------
Epoch2/9
----------
Epoch3/9
----------
Epoch4/9
----------
Epoch5/9
----------
Epoch6/9
----------
Epoch7/9
----------
Epoch8/9
----------
Epoch9/9
----------


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


train Loss:  0.0175 Acc: 0.0000
train Loss:  0.0325 Acc: 0.0000
train Loss:  0.0440 Acc: 0.0100
train Loss:  0.0572 Acc: 0.0200
train Loss:  0.0659 Acc: 0.0400
train Loss:  0.0734 Acc: 0.0600
train Loss:  0.0790 Acc: 0.0800
train Loss:  0.0988 Acc: 0.0900
train Loss:  0.1265 Acc: 0.0900
train Loss:  0.1461 Acc: 0.0900
train Loss:  0.1638 Acc: 0.0900
train Loss:  0.1742 Acc: 0.1100
train Loss:  0.1898 Acc: 0.1200
train Loss:  0.2074 Acc: 0.1200
train Loss:  0.2223 Acc: 0.1300
train Loss:  0.2342 Acc: 0.1500
train Loss:  0.2545 Acc: 0.1600
train Loss:  0.2661 Acc: 0.1800
train Loss:  0.2796 Acc: 0.1900
train Loss:  0.2917 Acc: 0.2000
train Loss:  0.3030 Acc: 0.2100
train Loss:  0.3214 Acc: 0.2200
train Loss:  0.3434 Acc: 0.2200
train Loss:  0.3623 Acc: 0.2200
train Loss:  0.3759 Acc: 0.2300
train Loss:  0.3862 Acc: 0.2500
train Loss:  0.3968 Acc: 0.2700
train Loss:  0.4196 Acc: 0.2700
train Loss:  0.4344 Acc: 0.2800
train Loss:  0.4474 Acc: 0.2900
train Loss:  0.4663 Acc: 0.2900
train Lo

In [ ]:
torch.save(model,'/content/drive/MyDrive/trained')

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Define the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the saved model
model = torch.load('/content/drive/MyDrive/trained')
model = model.to(device)
model.eval()

# Define the transformations for the new image
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
# Load the new image
image_path_1 = '/content/236A7806.JPG'
image = Image.open(image_path_1)

# Preprocess the image
input_image = transform(image).unsqueeze(0).to(device)

# Perform inference
with torch.no_grad():
    output = model(input_image)
    probabilities = torch.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

# Convert class index to class label (assuming class 0 is "lemon" and class 1 is "melon")
class_labels = ["Anju", "Zanvi"]
predicted_label = class_labels[predicted_class]

print("Predicted class:", predicted_label)
print("Probabilities:", probabilities)


Predicted class: Zanvi
Probabilities: tensor([[0.3051, 0.6949]], device='cuda:0')


In [ ]:
# Load the new image
image_path_1 = '/content/DSC_0319.JPG'
image = Image.open(image_path_1)

# Preprocess the image
input_image = transform(image).unsqueeze(0).to(device)

# Perform inference
with torch.no_grad():
    output = model(input_image)
    probabilities = torch.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()


class_labels = ["Anju", "Zanvi"]
predicted_label = class_labels[predicted_class]

print("Predicted class:", predicted_label)
print("Probabilities:", probabilities)


Predicted class: Anju
Probabilities: tensor([[0.5423, 0.4577]], device='cuda:0')


In [ ]:
# Load the new image
image_path_1 = '/content/236A7806.JPG'
image = Image.open(image_path_1)

# Preprocess the image
input_image = transform(image).unsqueeze(0).to(device)

# Perform inference
with torch.no_grad():
    output = model(input_image)
    probabilities = torch.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()


class_labels = ["Anju", "Zanvi"]
predicted_label = class_labels[predicted_class]

print("Predicted class:", predicted_label)
print("Probabilities:", probabilities)

Predicted class: Zanvi
Probabilities: tensor([[0.3051, 0.6949]], device='cuda:0')


In [ ]:
# Load the new image
image_path_1 = '/content/DSC_0634.JPG'
image = Image.open(image_path_1)

# Preprocess the image
input_image = transform(image).unsqueeze(0).to(device)

# Perform inference
with torch.no_grad():
    output = model(input_image)
    probabilities = torch.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()


class_labels = ["Anju", "Zanvi"]
predicted_label = class_labels[predicted_class]

print("Predicted class:", predicted_label)
print("Probabilities:", probabilities)

Predicted class: Zanvi
Probabilities: tensor([[0.3521, 0.6479]], device='cuda:0')
